In [49]:
import pandas as pd
import csv
from collections import defaultdict
from datetime import datetime
import requests
import bs4
import re
import dateutil
import warnings

In [50]:
# Initialize dictionaries to store the last message of the day for each sender
last_message_of_day_ellen_susan = {}
last_message_of_day_grant = {}

def data_reader(input_file, output_file, time_structure, last_messages_by_date, row_number, sender):
    """
    Reads data from an input CSV file, processes it to find the last message of each day for a specific sender,
    and writes the results to an output CSV file.

    Args:
    input_file (str): Path to the input CSV file.
    output_file (str): Path to the output CSV file.
    time_structure (str): Datetime format to be used for parsing.
    last_messages_by_date (dict): Dictionary to store the last message of each day.
    row_number (int): Index of the column to check if the message contains the name Tom as these are the messages that contain wordle scores.
    sender (str): Name of the sender to customize headers.

    Returns:
    None
    """

    def is_valid_datetime(date_str):
        """Check if the given string is a valid datetime according to the specified format."""
        try:
            datetime.strptime(date_str, time_structure)
            return True
        except ValueError:
            return False
    
    # Read the input CSV file
    with open(input_file, 'r', newline='', encoding='latin-1') as inp:
        reader = csv.reader(inp)
        header = next(reader)  # Skip the header row

        for row in reader:
            # Check if the message is from 'Tom'
            if "tom " in row[row_number].lower():
                datetime_str = row[1]  # Extract the datetime string
                if is_valid_datetime(datetime_str):
                    timestamp = datetime.strptime(datetime_str, time_structure)
                    date_only = timestamp.date()

                    # Update with the last message of the day
                    if date_only not in last_messages_by_date or last_messages_by_date[date_only][0] < timestamp:
                        last_messages_by_date[date_only] = (timestamp, row)

    # Write to the output CSV file
    with open(output_file, 'w', newline='', encoding='latin-1') as outp:
        writer = csv.writer(outp)

        # Write custom headers based on the sender
        if sender == 'grant':
            custom_header = ['Sender', 'Date', 'Text', 'Blank']
            writer.writerow(custom_header)
        else:
            writer.writerow(header)  # Use existing header for 'ellen_susan'

        # Write the last message of each day to the output file
        for date in sorted(last_messages_by_date.keys()):
            writer.writerow(last_messages_by_date[date][1])



In [51]:
def prepare_dataframe(table_name, date_name, message_name):
    """
    Prepare a DataFrame by selecting relevant columns and extracting date components.

    Args:
    table_name (DataFrame): The original DataFrame.
    date_name (str): The name of the date column.
    message_name (str): The name of the message column.

    Returns:
    DataFrame: A DataFrame with extracted date components.
    """
    df = table_name[[date_name, message_name]].copy()
    df['Message Date'] = pd.to_datetime(df[date_name])
    df['Year'] = df['Message Date'].dt.year
    df['Month'] = df['Message Date'].dt.month
    df['Day'] = df['Message Date'].dt.day
    df['Time'] = df['Message Date'].dt.time
    df['Message Date'] = df['Message Date'].dt.date 
    df[message_name] = df[message_name].str.replace(r'(?i)\bfail\b', '7', regex=True)

    return df

def split_text_by_sender(df, message_name, sender_name):
    """
    Split the text in the message column based on the sender's name.

    Args:
    df (DataFrame): The DataFrame with the message column.
    message_name (str): The name of the message column.
    sender_name (str): The name of the sender.

    Returns:
    DataFrame: The DataFrame with exploded text based on sender's name.
    """
    if sender_name == 'ellen_susan':
        df['Text'] = df[message_name].apply(lambda x: x.split('\n') if isinstance(x, str) else [])
    elif sender_name == 'grant':
        df['Text'] = df[message_name].apply(lambda x: re.split(r'(?<=\d)\s', x) if isinstance(x, str) else [])

    df = df.explode('Text').reset_index(drop=True)
    return df

def extract_player_score(df):
    """
    Extract player's name and score from the 'Text' column.

    Args:
    df (DataFrame): The DataFrame with the 'Text' column.

    Returns:
    DataFrame: The DataFrame with extracted player names and scores.
    """
    df['Player'] = df['Text'].str.extract(r'^([A-Za-z]+)')
    df['Score'] = df['Text'].str.extract(r'(\d+\.?\d*)$')
    df['Fail'] = (df['Score'] == '7').astype(int)  # Mark fails where 'Score' is NaN

    # Fill missing player names with 'National Average'
    df['Player'] = df['Player'].fillna('National Average')

    return df

def standardize_player_names(df):
    """
    Standardize player names to ensure consistency.

    Args:
    df (DataFrame): The DataFrame with the 'Player' column.

    Returns:
    DataFrame: The DataFrame with standardized player names.
    """
    df['Player'] = df['Player'].replace({
        'Idabelle': 'Isabelle',
        'Isabele': 'Isabelle',
        'Jen': 'Jennifer',
        'Metedith': 'Meredith',
        'NA': 'National Average'
    })
    return df

def map_words(df):
    """
    Map start, secondary, and tertiary words to players.

    Args:
    df (DataFrame): The DataFrame with the 'Player' column.

    Returns:
    DataFrame: The DataFrame with mapped words based on player names.
    """
    start_words = {
        'Beth': 'great',
        'smitty': 'audio',
        'grant': 'grant',
        'pete': 'route',
        'kate': 'ascot',
        'Lucy': 'weird',
        'Meredith': 'least',
        'slim': 'games'
    }

    secondary_words = {
        'Beth': 'pious',
        'Lucy': 'nasty'
    }

    tertiary_words = {
        'Lucy': 'cough'
    }

    df['Start_Words'] = df['Player'].map(start_words)
    df['secondary_words'] = df['Player'].map(secondary_words)
    df['tertiary_words'] = df['Player'].map(tertiary_words)

    return df

def proper_format(date_name, message_name, table_name, sender_name):
    """
    Process and format the DataFrame according to the specified transformations.

    Args:
    date_name (str): The name of the date column.
    message_name (str): The name of the message column.
    table_name (DataFrame): The original DataFrame.
    sender_name (str): The name of the sender.

    Returns:
    DataFrame: The fully processed and formatted DataFrame.
    """
    df = prepare_dataframe(table_name, date_name, message_name)
    df = split_text_by_sender(df, message_name, sender_name)
    df = extract_player_score(df)
    df = standardize_player_names(df)
    df = map_words(df)
    
    # Rearrange the columns to place them in the desired order
    df = df[['Message Date', 'Year', 'Month', 'Day', 'Time', message_name, 'Player', 'Score', 'Fail', 'Start_Words', 'secondary_words', 'tertiary_words']]

    # Display the first few rows of the DataFrame to verify the changes
    print(df.head(5))

    return df



In [52]:
#call the data reader function to process Ellen and Susans text messages
data_reader('wordle_texts.csv', 'filtered_wordle_scores_per_day.csv', '%Y-%m-%d %H:%M:%S', last_message_of_day_ellen_susan , 12, 'ellen_susan')

In [53]:
ellen_susan = pd.read_csv('filtered_wordle_scores_per_day.csv') # save the dataframe to an object ellen_susan


In [54]:
#call the data reader function to process grant text messages
data_reader('grant_texts.csv', 'grant_filtered_words.csv', '%b %d, %Y %I:%M:%S %p', last_message_of_day_grant, 2, 'grant')

In [55]:
grant = pd.read_csv('grant_filtered_words.csv')
grant.head(10)

,Sender,Date,Text,Blank
0,Sent,"Dec 31,\n1969\n6:59:59\nPM",Tom 5 NA = 3.2 Grant 3,NaN
1,Sent,"Sep 21,\n2023\n6:32:02\nPM",Grant 5 Lucy 5 Marcos 3 Pete 3 Tom 3,NaN
2,Sent,"Sep 22,\n2023\n3:40:21\nPM",Grant 3 Marcos 3 Pete 3 Tom 3,NaN
3,Sent,"Sep 23,\n2023\n5:55:40\nPM",Grant 4 Marcos 4 Tom 4 Pete 3,NaN
4,Sent,"Sep 24,\n2023\n7:57:52\nPM",Marcos 5 Grant 3 Tom 3 Beth 2,NaN
5,Sent,"Sep 25,\n2023\n11:04:54\nAM",Beth 5 Jen 4 Lucy 4 Grant 3 Tom 3,NaN
6,Sent,"Sep 26,\n2023\n9:13:39\nAM",Jen 4 Beth 3 Grant 3 Tom 3,NaN
7,Sent,"Sep 27,\n2023\n8:20:22\nPM",Beth 5 Seth 5 Tom 4 Ellen 3 Grant 3 Jen 3 Marc...,NaN
8,Sent,"Sep 28,\n2023\n9:34:17\nPM",Beth 5 Marcos 4 Pete 4 Ellen 3 Grant 3 Jennife...,NaN
9,Received,"Sep 29,\n2023\n7:52:59\nPM",From Benson: Yeah if Tom comes and joins our s...,NaN


In [56]:
# Call the proper format function with the appropriate arguments for ellen_susan data
expanded_ellen_susan = proper_format('Message Date', 'Text', ellen_susan, 'ellen_susan')


  Message Date  Year  Month  Day      Time     Text    Player Score  Fail  \
0   2023-07-19  2023      7   19  22:36:36   Beth 5      Beth     5     0   
1   2023-07-19  2023      7   19  22:36:36  Ellen 5     Ellen     5     0   
2   2023-07-19  2023      7   19  22:36:36   Lucy 4      Lucy     4     0   
3   2023-07-19  2023      7   19  22:36:36    Tom 4       Tom     4     0   
4   2023-07-19  2023      7   19  22:36:36    Jen 3  Jennifer     3     0   

  Start_Words secondary_words tertiary_words  
0       great           pious            NaN  
1         NaN             NaN            NaN  
2       weird           nasty          cough  
3         NaN             NaN            NaN  
4         NaN             NaN            NaN  


In [57]:
# Call the proper formatfunction with the appropriate arguments for grant data
expanded_grant = proper_format('Date', 'Text', grant, 'grant')

  Message Date  Year  Month  Day      Time      Text            Player Score  \
0   1969-12-31  1969     12   31  18:59:59     Tom 5               Tom     5   
1   1969-12-31  1969     12   31  18:59:59  NA = 3.2  National Average   3.2   
2   1969-12-31  1969     12   31  18:59:59   Grant 3             Grant     3   
3   2023-09-21  2023      9   21  18:32:02   Grant 5             Grant     5   
4   2023-09-21  2023      9   21  18:32:02    Lucy 5              Lucy     5   

   Fail Start_Words secondary_words tertiary_words  
0     0         NaN             NaN            NaN  
1     0         NaN             NaN            NaN  
2     0         NaN             NaN            NaN  
3     0         NaN             NaN            NaN  
4     0       weird           nasty          cough  


/var/folders/pk/3dlwb56x1gz391w3ddxwfqrw0000gn/T/ipykernel_5918/3580904977.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Message Date'] = pd.to_datetime(df[date_name])


In [58]:
#merge both text data frames into a single dataframe
texts = pd.concat([expanded_grant, expanded_ellen_susan], axis=0, ignore_index=True)
texts.head(10)

,Message Date,Year,Month,Day,Time,Text,Player,Score,Fail,Start_Words,secondary_words,tertiary_words
0,1969-12-31,1969,12,31,18:59:59,Tom 5,Tom,5,0,NaN,NaN,NaN
1,1969-12-31,1969,12,31,18:59:59,NA = 3.2,National Average,3.2,0,NaN,NaN,NaN
2,1969-12-31,1969,12,31,18:59:59,Grant 3,Grant,3,0,NaN,NaN,NaN
3,2023-09-21,2023,9,21,18:32:02,Grant 5,Grant,5,0,NaN,NaN,NaN
4,2023-09-21,2023,9,21,18:32:02,Lucy 5,Lucy,5,0,weird,nasty,cough
5,2023-09-21,2023,9,21,18:32:02,Marcos 3,Marcos,3,0,NaN,NaN,NaN
6,2023-09-21,2023,9,21,18:32:02,Pete 3,Pete,3,0,NaN,NaN,NaN
7,2023-09-21,2023,9,21,18:32:02,Tom 3,Tom,3,0,NaN,NaN,NaN
8,2023-09-22,2023,9,22,15:40:21,Grant 3,Grant,3,0,NaN,NaN,NaN
9,2023-09-22,2023,9,22,15:40:21,Marcos 3,Marcos,3,0,NaN,NaN,NaN


Next pull wordle answer data from the web to see each days word

In [59]:

# URL of the webpage containing Wordle answers
URL = 'https://wordfinder.yourdictionary.com/wordle/answers/'

# Send a GET request to the URL to retrieve the webpage content
req = requests.get(URL)

# Parse the HTML content of the webpage using BeautifulSoup
soup = bs4.BeautifulSoup(req.text, 'html.parser')

# Define the headers for the CSV file
HEADERS = ['Date', 'Wordle', 'Answer']

# Initialize an empty list to store all rows of data
all_rows = []

# Loop over the tables on the webpage. Currently set to 14, but may need to be adjusted as new months are added to the site
for table_index in range(14):
    try:
        # Find the table on the webpage based on the current index
        table = soup.find_all('table')[table_index]
        rows = table.find_all('tr')  # Get all rows from the table

        # Start looping from the second row (index 1) because the first row usually contains headers
        for row in rows[1:]:
            cells = row.find_all('td')  # Get all cells in the row
            Date = cells[0].text.strip()  # Extract the date from the first cell (Note: The year is not grabbed)
            Wordle = cells[1].text.strip()  # Extract the Wordle number from the second cell
            Answer = cells[2].text.strip()  # Extract the answer from the third cell
            all_rows.append([Date, Wordle, Answer])  # Append the extracted data as a list to all_rows

    except IndexError:
        # Print a message if a table with the current index does not exist on the webpage
        print(f"Table {table_index} does not exist on the page.")

# Write the extracted data to a CSV file named 'wordle_answers.csv'
with open('wordle_answers.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(HEADERS)  # Write the headers to the CSV file
    writer.writerows(all_rows)  # Write all rows of data to the CSV file

# Print a confirmation message that the data has been written to the CSV file
print(f"Data from tables 0 through 13 has been written to 'wordle_answers.csv'.")



Data from tables 0 through 13 has been written to 'wordle_answers.csv'.


In [60]:
answers = pd.read_csv('wordle_answers.csv') #save csv to variable
answers = answers.drop(0).reset_index(drop=True)

print(answers)

        Date  Wordle Answer
0    Aug. 13    1151  NEIGH
1    Aug. 12    1150  SKIFF
2    Aug. 11    1149  SCONE
3    Aug. 10    1148  MEDIC
4    Aug. 09    1147  OUNCE
..       ...     ...    ...
405  Jul. 05     746  VENOM
406  Jul. 04     745  IRATE
407  Jul. 03     744  HOTEL
408  Jul. 02     743  MOSSY
409  Jul. 01     742  BLEEP

[410 rows x 3 columns]


In [61]:
### this code chunk adjusts the date column so it follows the same format as our date column in our merge table
month_map = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}

# Function to transform the date format
def transform_date(date_str, wordle_number):
    # Remove the period after the month and split by space
    parts = date_str.replace('.', '').split()
    # Ensure parts has exactly two elements
    if len(parts) != 2:
        raise ValueError(f"Date string format is incorrect: {date_str}")
    # Map the month name to its corresponding number
    month_number = month_map[parts[0]]
    # Determine the year based on the Wordle Answer number
    year = 2024 if wordle_number >= 926 else 2023 # looked up this value from website
    return f"{year}-{month_number}-{parts[1]}"

# Apply the function to the 'Date' column
answers['Date'] = answers.apply(lambda row: transform_date(row['Date'], row['Wordle']), axis=1)

# Display the transformed DataFrame
print(answers)

           Date  Wordle Answer
0    2024-08-13    1151  NEIGH
1    2024-08-12    1150  SKIFF
2    2024-08-11    1149  SCONE
3    2024-08-10    1148  MEDIC
4    2024-08-09    1147  OUNCE
..          ...     ...    ...
405  2023-07-05     746  VENOM
406  2023-07-04     745  IRATE
407  2023-07-03     744  HOTEL
408  2023-07-02     743  MOSSY
409  2023-07-01     742  BLEEP

[410 rows x 3 columns]


In [62]:
# Initialize an empty dictionary to store date-answer pairs
answers_dictionary = {}

# Loop through each row in the DataFrame 'answers'
for i in range(len(answers)):
    date = answers.iloc[i, 0]  # Extract the date from the first column (index 0)
    answer = answers.iloc[i, 2]  # Extract the answer from the third column (index 2)
    
    # Add the date and answer as a key-value pair in the dictionary
    answers_dictionary[date] = answer

# Convert the date strings in the dictionary keys to datetime objects for easier manipulation
answers_dictionary = {datetime.strptime(k, '%Y-%m-%d'): v for k, v in answers_dictionary.items()}


In [63]:
texts['wordle_answers'] = texts['Message Date'].map(answers_dictionary)
print(texts)

     Message Date  Year  Month  Day      Time      Text            Player  \
0      1969-12-31  1969     12   31  18:59:59     Tom 5               Tom   
1      1969-12-31  1969     12   31  18:59:59  NA = 3.2  National Average   
2      1969-12-31  1969     12   31  18:59:59   Grant 3             Grant   
3      2023-09-21  2023      9   21  18:32:02   Grant 5             Grant   
4      2023-09-21  2023      9   21  18:32:02    Lucy 5              Lucy   
...           ...   ...    ...  ...       ...       ...               ...   
3458   2024-07-25  2024      7   25  10:39:00  NA = 3.5  National Average   
3459   2024-07-25  2024      7   25  10:39:00   Grant 3             Grant   
3460   2024-07-25  2024      7   25  10:39:00     Jen 3          Jennifer   
3461   2024-07-25  2024      7   25  10:39:00     Tom 3               Tom   
3462   2024-07-25  2024      7   25  10:39:00  Marcos 2            Marcos   

     Score  Fail Start_Words secondary_words tertiary_words wordle_answers 

In [64]:
texts.to_csv('final_texts.csv', index=False)